In [29]:
from PIL import Image, ImageDraw, ImageFont
import os
import piexif
from datetime import datetime

def add_watermark(image_path, text, save_as=None):
    original = Image.open(image_path)
    txt = Image.new('RGBA', original.size, (255,255,255,0))
    draw = ImageDraw.Draw(txt)
    font = ImageFont.truetype("arial.ttf", 20)  # Ensure this font path is correct

    # Estimate the text position for bottom center alignment
    text_x = original.width // 2  # Center the text horizontally
    text_y = original.height - 30  # Position the text towards the bottom

    # Adjust the text position slightly if needed to ensure it's centered
    text_position = (text_x, text_y)

    # Using anchor='mm' to center the text based on the text_x calculation
    draw.text(text_position, text, fill=(255,255,255,128), font=font, anchor="mm")
    watermarked = Image.alpha_composite(original.convert('RGBA'), txt).convert('RGB')
    
    if save_as:
        watermarked.save(save_as, 'JPEG')
    else:
        watermarked.save(image_path, 'JPEG')



def get_camera_info(exif_data):
    camera_info = {
        'make': exif_data['0th'].get(piexif.ImageIFD.Make, b'Unknown').decode('utf-8'),
        'model': exif_data['0th'].get(piexif.ImageIFD.Model, b'Unknown').decode('utf-8'),
        'iso': str(exif_data['Exif'].get(piexif.ExifIFD.ISOSpeedRatings, 'Unknown')),
        'fstop': 'Unknown',
        'shutterspeed': 'Unknown',
        'datetime': exif_data['Exif'].get(piexif.ExifIFD.DateTimeOriginal, b'Unknown').decode('utf-8')
    }

    # Parsing FNumber for f-stop
    if piexif.ExifIFD.FNumber in exif_data['Exif']:
        f_number = exif_data['Exif'][piexif.ExifIFD.FNumber]
        fstop_value = f_number[0] / f_number[1]
        camera_info['fstop'] = f"f/{fstop_value}"

    # Parsing ExposureTime for shutter speed
    if piexif.ExifIFD.ExposureTime in exif_data['Exif']:
        exposure_time = exif_data['Exif'][piexif.ExifIFD.ExposureTime]
        shutter_speed_value = exposure_time[0] / exposure_time[1]
        # To handle long exposure times that are not fractions
        if shutter_speed_value < 1:
            camera_info['shutterspeed'] = f"1/{round(1/shutter_speed_value)} sec"
        else:
            camera_info['shutterspeed'] = f"{round(shutter_speed_value, 2)} sec"

    if camera_info['datetime'] != 'Unknown':
        year_taken = datetime.strptime(camera_info['datetime'], '%Y:%m:%d %H:%M:%S').year
    else:
        year_taken = 'Unknown'
    
    return camera_info, year_taken


def process_images(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            exif_data = piexif.load(image_path)
            camera_info, year_taken = get_camera_info(exif_data)

            watermark_text = (f"© Charles Lehnen {year_taken}, {camera_info['make']} {camera_info['model']}, "
                              f"ISO {camera_info['iso']}, {camera_info['fstop']}, {camera_info['shutterspeed']}")
            
            add_watermark(image_path, watermark_text, save_as=f"{image_path}_watermarked.jpg")

# Adjust the path to your images folder
process_images('../images/no_watermark')
